# Inference

History
- 4/9
   - implement ensemble
- 4/10
   - implement TTA

In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys, cv2, glob, datetime
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from classifier import *

ensemble_models = ['densenet201','resnet101', 'xception', 'resnet101v2', 'inception_resnet_v2','resnet152v2','densenet169']
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    
#NASNetLarge(include_top=False, input_shape=(331,331,3))     


### 1. Prepare data

In [ ]:

tr_floder = '/mnt/hdd1/Data/Competition/orchid/training'
df_tr = pd.DataFrame({'filepath':glob.glob(os.path.join(tr_floder,'*.jpg'))})
df_tr['filename'] = df_tr['filepath'].map(lambda x: os.path.basename(x))
df_tr['key']=df_tr['filename']

#新增training aug 0412
aug_folder='/mnt/hdd1/Data/Competition/orchid/training_aug'
df_aug = pd.DataFrame({'filepath':glob.glob(os.path.join(aug_folder,'*.jpg'))})
df_aug['filename'] = df_aug['filepath'].map(lambda x: os.path.basename(x))
df_aug['key']=df_aug['filename'].map(lambda x: x[4:])

#TODO: 新增test_public
#TODO: 新增test_private


### 2. Inference

In [ ]:
clf = Classifier()
clf.model_dir = '/mnt/hdd1/Model/Competition/orchid'
clf.cls_map_path = 'model/model_cls_map.pkl'
clf.model_ver = '0412v1'
clf.ensemble_models = ensemble_models
clf.tta_n = 3

In [ ]:

df_tr = clf.inference(df_tr, output_prefix='tr')
df_aug = clf.inference(df_aug, output_prefix='traug')

### Generate submit.csv

### Acc evaluate

In [ ]:
clf.label_path = '/mnt/hdd1/Data/Competition/orchid/training/label.csv'

clf.evaluate_acc(df_tr)
clf.evaluate_acc(df_aug)


***

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
tiral_ver='0410v2'
model_dir = '/mnt/hdd1/Model/Competition/orchid'
model_name = 'densenet201'
gen_df = get_data_gen(model_name)
inv_map = {v: k for k, v in gen_df.class_indices.items()}

model_path = os.path.join(model_dir, f'{tiral_ver}/{model_name}_{tiral_ver}_full.h5')
model = load_model(model_path, compile=False)
print(f'==== {model_name} ===')
pred = model.predict(gen_df, verbose=1)
pred_score = np.max(pred, axis=1)
pred_label = np.argmax(pred, axis=1)
pred_class = [inv_map[x] for x in pred_label]
df[f'{model_name}_pred_score'] = pred_score
df[f'{model_name}_pred_class'] = pred_class
df[f'{model_name}_pred_class'] = df[f'{model_name}_pred_class'].astype(int)
acc = accuracy_score(df['category'], df[f'{model_name}_pred_class'])
print(f'{model_name}:{acc:.2f}')

